# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Bañados  ->  E. Bañados  |  ['E. Bañados']
C. Mao  ->  C. Mao  |  ['C. Mao']
Arxiv has 44 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2401.10328


extracting tarball to tmp_2401.10328...

 done.


Found 123 bibliographic references in tmp_2401.10328/fireqsos.bbl.
Retrieving document from  https://arxiv.org/e-print/2401.10717


extracting tarball to tmp_2401.10717...

 done.


/tmp/ipykernel_2261/3009462391.py:49: LatexWarning: 2401.10717 did not run properly
'utf-8' codec can't decode byte 0xa8 in position 25952: invalid start byte
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.10328-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.10328) | **Chronicling the reionization history at $6\lesssim z \lesssim 7$ with  emergent quasar damping wings**  |
|| D. Ďurovčíková, et al. -- incl., <mark>E. Bañados</mark> |
|*Appeared on*| *2024-01-22*|
|*Comments*| *28 pages, 14 figures*|
|**Abstract**| The spectra of high-redshift ($z\gtrsim 6$) quasars contain valuable information on the progression of the Epoch of Reionization (EoR). At redshifts $z<6$, the observed Lyman-series forest shows that the intergalactic medium (IGM) is nearly ionized, while at $z>7$ the observed quasar damping wings indicate high neutral gas fractions. However, there remains a gap in neutral gas fraction constraints at $6\lesssim z \lesssim 7$ where the Lyman series forest becomes saturated but damping wings have yet to fully emerge. In this work, we use a sample of 18 quasar spectra at redshifts $6.0<z<7.1$ to close this gap. We apply neural networks to reconstruct the quasars' continuum emission around the partially absorbed Lyman $\alpha$ line to normalize their spectra, and stack these continuum-normalized spectra in three redshift bins. To increase the robustness of our results, we compare the stacks to a grid of models from two hydrodynamical simulations, ATON and CROC, and we measure the volume-averaged neutral gas fraction, $\bar{x}_{\rm HI}$, while jointly fitting for the mean quasar lifetime, $t_{\rm Q}$, for each stacked spectrum. We chronicle the evolution of neutral gas fraction using the ATON (CROC) models as follows: $\bar{x}_{\rm HI} = 0.21_{-0.07}^{+0.17}$ ($\bar{x}_{\rm HI} = 0.10_{<10^{-4}}^{+0.73}$) at $\langle z \rangle =6.10$, $\bar{x}_{\rm HI} = 0.21_{-0.07}^{+0.33}$ ($\bar{x}_{\rm HI} =0.57_{-0.47}^{+0.26}$) at $\langle z \rangle =6.46$, and $\bar{x}_{\rm HI} = 0.37_{-0.17}^{+0.17}$ ($\bar{x}_{\rm HI} =0.57_{-0.21}^{+0.26}$) at $\langle z \rangle =6.87$. At the same time we constrain the average quasar lifetime to be $t_{\rm Q} \lesssim 7\ {\rm Myr}$ across all redshift bins, in good agreement with previous studies. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.10717-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.10717) | **BSEC method for unveiling open clusters and its application to Gaia DR3:  83 new clusters**  |
|| Z. Li, <mark>C. Mao</mark> |
|*Appeared on*| *2024-01-22*|
|*Comments*| *21 pages, 13 figures, 5 tables. Submitted to RAA*|
|**Abstract**| Open clusters (OCs) are common in the Milky Way, but most of them remain undiscovered. There are numerous techniques, including some machine-learning algorithms, available for the exploration of OCs. However, each method has its own limitations and therefore, different approaches to discovering OCs hold significant value. We develop a comprehensive approach method to automatically explore the data space and identify potential OC candidates with relatively reliable membership determination. This approach combines the techniques of HDBSCAN, GMM, and a novel cluster member identification technique, 2-color constraint. The new method exhibits efficiency in detecting OCs while ensuring precise determination of cluster memberships. Because the main feature of this technique is to add an extra constraint for the members of cluster candidates using the homogeneity of color excess, comparing to typical blind search codes, it is called Blind Search-Extra Constraint (BSEC) method. It is successfully applied to the Gaia Data Release 3, and 83 new OCs with CMDs similar to stellar isochrones are found. In addition, this study reports 621 new OC candidates including at least the main sequence or red giant branch. It is shown that BSEC technique can discard some false negatives of previous works, which takes about 3 percentage of known clusters. It shows that color excess (or 2-color) constraint is useful for removing fake cluster member stars and getting more precise CMDs. It makes the CMDs of 15 percent clusters clearer (in particular for the region near turnoff) and therefore is helpful for CMD and stellar population studies. Keywords: galaxy: stellar content, open clusters and associations; stars: fundamental parameters |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xa8 in position 25952: invalid start byte</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2401.10328.md
    + _build/html/tmp_2401.10328/./2dlikelihood__600_10000_final_P-GADGET-3_composite.png
    + _build/html/tmp_2401.10328/./2dlikelihood__600_10000_final_CROC_composite.png
    + _build/html/tmp_2401.10328/./6.87_600_10000_final_noisemodel_0.36_xT_1e6yr.png
    + _build/html/tmp_2401.10328/./z-stacks_paper_main.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\msol}{M_\odot}$
$\newcommand{\vxhi}{\bar{x}_{\rm HI}}$
$\newcommand{\logtq}{\log{t_{\rm Q}}}$
$\newcommand{\zbar}{\langle z \rangle }$
$\newcommand{\chimp}{{\rm c Mpc}/h}$
$\newcommand{\lya}{Ly\alpha }$
$\newcommand{\lyb}{Ly\beta }$
$\newcommand{\lyc}{Ly\gamma}$
$\newcommand{\ciisf}{[C II]}$
$\newcommand{\cii}{[C{\small II}]}$
$\newcommand{\civ}{C{\small IV}}$
$\newcommand{\mgii}{Mg{\small II}}$
$\newcommand{\oiii}{[O{\small III}]}$
$\newcommand{\ddu}[1]{\texttt{\color{red}[DD: #1]}}$</div>



<div id="title">

# Chronicling the reionization history at $6\lesssim z \lesssim 7$ with emergent quasar damping wings

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2401.10328-b31b1b.svg)](https://arxiv.org/abs/2401.10328)<mark>Appeared on: 2024-01-22</mark> -  _28 pages, 14 figures_

</div>
<div id="authors">

D. {. D. c}{í}kov{á}, et al.

</div>
<div id="abstract">

**Abstract:** The spectra of high-redshift ( $z\gtrsim 6$ ) quasars contain valuable information on the progression of the Epoch of Reionization (EoR). At redshifts $z<6$ , the observed Lyman-series forest shows that the intergalactic medium (IGM) is nearly ionized, while at $z>7$ the observed quasar damping wings indicate high neutral gas fractions. However, there remains a gap in neutral gas fraction constraints at $6\lesssim z \lesssim 7$ where the Lyman series forest becomes saturated but damping wings have yet to fully emerge.In this work, we use a sample of 18 quasar spectra at redshifts $6.0<z<7.1$ to close this gap. We apply neural networks to reconstruct the quasars' continuum emission around the partially absorbed Lyman $\alpha$ line to normalize their spectra, and stack these continuum-normalized spectra in three redshift bins. To increase the robustness of our results, we compare the stacks to a grid of models from two hydrodynamical simulations, ATON and CROC, and we measure the volume-averaged neutral gas fraction, $\vxhi$ , while jointly fitting for the mean quasar lifetime, $t_{\rm Q}$ , for each stacked spectrum. We chronicle the evolution of neutral gas fraction using the ATON (CROC) models as follows: $\vxhi = 0.21_{-0.07}^{+0.17}$ ( $\vxhi = 0.10_{<10^{-4}}^{+0.73}$ ) at $\zbar =6.10$ , $\vxhi = 0.21_{-0.07}^{+0.33}$ ( $\vxhi =0.57_{-0.47}^{+0.26}$ ) at $\zbar =6.46$ , and $\vxhi = 0.37_{-0.17}^{+0.17}$ ( $\vxhi =0.57_{-0.21}^{+0.26}$ ) at $\zbar =6.87$ . At the same time we constrain the average quasar lifetime to be $t_{\rm Q} \lesssim 7 {\rm Myr}$ across all redshift bins, in good agreement with previous studies.

</div>

<div id="div_fig1">

<img src="tmp_2401.10328/./2dlikelihood__600_10000_final_P-GADGET-3_composite.png" alt="Fig10.1" width="50%"/><img src="tmp_2401.10328/./2dlikelihood__600_10000_final_CROC_composite.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** Maximum likelihood estimation with ATON models (top two rows) and CROC models (bottom two rows) for all three redshift bins. In the top panel, we show the 2D joint probability distribution evaluated for each combination of $\logtq$ and $\vxhi$, as well as the $1\sigma$ and $2\sigma$ contours as white dashed and dotted curves, respectively. Note that the parameter space where we do not have ATON models is hatched. In the bottom panel, we plot stacked spectrum for each bin along with the model corresponding to the maximum likelihood point on the 2D grid in the top panel (marked by a star). (*fig:fits*)

</div>
<div id="div_fig2">

<img src="tmp_2401.10328/./6.87_600_10000_final_noisemodel_0.36_xT_1e6yr.png" alt="Fig8" width="100%"/>

**Figure 8. -** Illustration of the forward modeling of uncertainties and model construction for both simulations (ATON in the top panel, CROC in the bottom panel). For a particular quasar and a combination of $\vxhi$ and $\logtq$ values, we draw one sightline at random from the corresponding simulation (first column from the left). We forward model the uncertainties by adding the smooth continuum prediction noise (second column), random measurement noise (third column), and finally the corresponding redshift error along the x-axis (fourth column; $\sigma_z = 0.01$ for illustration purposes). We repeat this sightline processing for all quasars within a given redshift bin ($N_{\rm QSO}$), and then stack $N_{\rm QSO}$ processed sightlines to form a sample stack (rightmost column). The whole procedure is repeated $10000$ times for the same combination of $\vxhi$ and $\logtq$ in order to calculate the final model and noise statistics that are used for inference. (*fig:noise*)

</div>
<div id="div_fig3">

<img src="tmp_2401.10328/./z-stacks_paper_main.png" alt="Fig2" width="100%"/>

**Figure 2. -** Stacked spectra of quasars in our sample in three distinct redshift bins show the emergence of damping wings between $z\sim 6$ and $z\sim 7$(thick colored curves, the location of $\lya$ is marked by the dashed black line). The number of quasars within a given redshift bin is noted in brackets, and the shaded region around the stacked spectra shows the variance in the mean flux. The thin colored lines show a higher-resolution ($\Delta v = 100 {\rm km/s}$) version of the three stacked spectra. (*fig:zstacks*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2401.10328"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

356  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
